# Deploy trained model

- Connect to the Workspace
- Get Experiment and the registered model
- Deploy using the Portal

In [ ]:
from azureml.core import Workspace, Experiment, Dataset

ws = Workspace.from_config()

exp = Experiment(workspace=ws, name='my-experiment')

In [ ]:
titanic_model = ws.models['titanic-sklearn-dctree']

### Use the AzureML UI

Deploy the model using the UI into ACI. We have a model that accepts rows with a combination of `String` and `float` types. Only models that are compatible with input `Array[float]` can be deployed as No-Code. We need a custom wrapper that transforms the payload into Pandas DataFrame. The `scripts/entry_script.py` is the wrapper.

- ACI is a Dev/Test solution for running a container-group.
- AKS (Kubernetes) is a production runtime (inference) solution

> Note: Only models with input `Array[float]` can be deployed as No-Code solutions!

**Use custom deploy Assets**
- Entry Script: `scripts/entry_script.py`
- Conda dependency file: `scripts/conda_env.py`

![Deploy with the UI](../../images/deploy_aci_using_ui.png)

In [ ]:
from azureml.core import Webservice

services = Webservice.list(ws)
for service in services:
    print(service.name)
    print(" - Scoring  : " + service.scoring_uri)
    print(" - API Descr: " + service.swagger_uri)
    print("")

- Check the Swagger URL: install a browser-plugin to see the API.
- Which endpoints are avaiable and what is the purpose of the GET-request? 
- What are the expected arguments of the /score POST endpoint?

## Call your service

Find your publicly exposed URL. By clicking on the endpoints.

- Endpoints
- Real-Time endpoints
- Click on your model

![copy_aci_url](../../images/copy_aci_url.png)


In [ ]:
from azureml.core import Dataset

predict_ds = Dataset.get_by_name(ws, name="titanic_predict_ds")
rows_to_predict = predict_ds.take(5).to_pandas_dataframe()
rows_to_predict

In [ ]:
# Convert into list[list[values]], as expected by the Rest API
test_values = rows_to_predict.values.tolist()

test_values[:1]

In [ ]:
import requests

res = requests.post(
    'http://YOUR-ACI-RUNTIME-UNIQUE-ID.westeurope.azurecontainer.io/score',
    json = {"data": test_values},
    headers = {'Content-Type':'application/json'}
)

res.raise_for_status()
res.json()

- Challenge 1: Deploy using code instead of the UI 